# नोटबुक 3: PP-OCR के साथ टेक्स्ट रिकॉग्निशन मॉडल को ट्रेनिंग देना 🏆

**उद्देश्य:** इस नोटबुक का उद्देश्य `1_Data_Preparation.ipynb` में तैयार की गई क्रॉप की गई शब्द छवियों का उपयोग करके एक टेक्स्ट रिकॉग्निशन मॉडल को प्रशिक्षित करना है। हम एक कस्टम कैरेक्टर डिक्शनरी बनाएंगे, कॉन्फ़िगरेशन को संशोधित करेंगे, और मॉडल को प्रशिक्षित करेंगे।

---

## चरण 1: परिवेश सेटअप
यह मान लिया जाता है कि आपने नोटबुक 2 से पहले ही PaddleOCR स्थापित और क्लोन कर लिया है।

In [ ]:
import os
import yaml

# PaddleOCR निर्देशिका में जाएँ
if os.path.basename(os.getcwd()) != 'PaddleOCR':
    %cd PaddleOCR

---

## चरण 2: कस्टम कैरेक्टर डिक्शनरी बनाना
एक रिकॉग्निशन मॉडल को आपके डेटासेट में मौजूद सभी अद्वितीय अक्षरों की एक सूची की आवश्यकता होती है।

In [ ]:
label_file = '../dataset/rec_label.txt'
unique_chars = set()

with open(label_file, 'r') as f:
    for line in f:
        try:
            _, text = line.strip().split('\t')
            for char in text:
                unique_chars.add(char)
        except ValueError:
            continue

dict_content = '\n'.join(sorted(list(unique_chars)))
dict_path = '../dataset/custom_dict.txt'
with open(dict_path, 'w') as f:
    f.write(dict_content)

print(f"कस्टम डिक्शनरी '{dict_path}' में सहेजी गई जिसमें {len(unique_chars)} अक्षर हैं।")
!cat {dict_path}

---

## चरण 3: रिकॉग्निशन कॉन्फ़िगरेशन को संशोधित करना
हम एक हल्के CRNN मॉडल के लिए कॉन्फ़िगरेशन को संशोधित करेंगे ताकि वह हमारी कस्टम डिक्शनरी और डेटा पथों का उपयोग करे।

In [ ]:
config_path = 'configs/rec/PP-OCRv3/en_PP-OCRv3_rec.yml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

config['Global']['save_model_dir'] = './output/rec_crnn'
config['Global']['epoch_num'] = 100
config['Global']['character_dict_path'] = '../dataset/custom_dict.txt'

config['Optimizer']['lr']['learning_rate'] = 0.0005
config['Train']['dataset']['data_dir'] = '../dataset/'
config['Train']['dataset']['label_file_list'] = ['../dataset/rec_label_train.txt']

config['Eval']['dataset']['data_dir'] = '../dataset/'
config['Eval']['dataset']['label_file_list'] = ['../dataset/rec_label_val.txt']

# आउटपुट क्लास की संख्या को हमारी डिक्शनरी से मेल खाने के लिए बदलें (+1 ब्लैंक, +1 CTC के लिए)
config['Head']['out_channels'] = len(unique_chars) + 1 

custom_config_path = 'configs/rec/my_custom_rec_config.yml'
with open(custom_config_path, 'w') as f:
    yaml.dump(config, f)

print(f"कस्टम रिकॉग्निशन कॉन्फ़िगरेशन '{custom_config_path}' में सहेजा गया।")

---

## चरण 4: रिकॉग्निशन ट्रेनिंग शुरू करना
अब हम अपनी नई कॉन्फ़िगरेशन फ़ाइल के साथ रिकॉग्निशन मॉडल को प्रशिक्षित करेंगे।

In [ ]:
!python tools/train.py -c configs/rec/my_custom_rec_config.yml

---

## चरण 5: प्रशिक्षित रिकॉग्निशन मॉडल के साथ अनुमान
ट्रेनिंग के बाद, हम एक एकल क्रॉप की गई शब्द छवि को पहचानने के लिए अपने मॉडल का उपयोग करेंगे।

In [ ]:
from paddleocr import PaddleOCR

# PaddleOCR को हमारे प्रशिक्षित रिकॉग्निशन मॉडल के साथ प्रारंभ करें
ocr_engine = PaddleOCR(
    det=False, # हम केवल रिकॉग्निशन का परीक्षण कर रहे हैं
    rec_model_dir='./output/rec_crnn/best_accuracy',
    rec_char_dict_path='../dataset/custom_dict.txt',
    use_gpu=True
)

img_path = '../dataset/word_crops/crop_0000.png'
result = ocr_engine.ocr(img_path, cls=False, det=False)

print("रिकॉग्निशन परिणाम:")
print(result)